# Using Defog.ai SQL-Eval with SQLite database

This notebook runs through the process of using the sql-eval repository with Sqlite database. It uses sample data from https://github.com/defog-ai/defog-data

### Clone repo and install requirements

In [ ]:
!git clone https://github.com/defog-ai/defog-data.git
!pip install -r /content/defog-data/requirements.txt

In [ ]:
!git clone https://github.com/defog-ai/sql-eval.git
!mv sql-eval/* .
!pip install -r requirements.txt

In [ ]:
!mv defog-data/defog_data .

In [ ]:
import os
import pandas

os.environ["OPENAI_API_KEY"] = "OPEN-API-KEY"

# NOTE

For the sake of this example, we will be using only 3 databases - academic, advising and atis. Hence modify `db_names` and `test_queries` variable in ***defog-data/translate_ddl_dialect.py*** to have only these 3 tables

In [ ]:
# modify translate_ddl_dialect.py to have only first 3 databases
# db_names = [
#     "academic",
#     "advising",
#     "atis",
# ]
# test_queries = [
#     ("academic", "writes"),
#     ("advising", "student_record"),
#     ("atis", "time_zone"),
# ]

### Create and populate SQLite databases

In [ ]:
!python /content/defog-data/translate_ddl_dialect.py --dialects sqlite


NOTE : In utils/creds.py change `"path_to_folder": f"/content/sqlite_dbs/"` for sqlite (or whatever the path of sqlite database is)


### Modify questions set to have only question for selected db

In [ ]:
import pandas as pd
questions_gen_sqlite = pd.read_csv("data/questions_gen_sqlite.csv")

In [ ]:
questions_gen_sqlite_trimmed = questions_gen_sqlite[questions_gen_sqlite['db_name'].isin(["academic","advising","atis"])]

In [ ]:
questions_gen_sqlite_trimmed.to_csv("questions_gen_sqlite_trimmed.csv", index=False)

### Run evaluation code

In [ ]:
!python main.py \
  -db sqlite \
  -q questions_gen_sqlite_trimmed.csv \
  -o results/openai_classic.csv \
  -g oa \
  -f prompts/prompt_openai.json \
  -m gpt-3.5-turbo


The evaluation results will be available in csv file within results folder

### Check data in SQLite databases

In [ ]:
import sqlite3
conn = sqlite3.connect('sqlite_dbs/academic.db')
c = conn.cursor()

sql_query="""
SELECT publication.year, AVG(publication.citation_num) AS average_citations FROM publication GROUP BY publication.year ORDER BY publication.year NULLS LAST;

"""

c.execute(sql_query)
print(c.fetchall())